In [1]:
    if __name__ == '__main__':
        %reload_ext importable
    from contextlib import contextmanager
    from fnmatch import fnmatch
    from functools import partial
    from importable import finder, unload
    from IPython import get_ipython, display
    from IPython.core.interactiveshell import InteractiveShell
    from mistune import Markdown, Renderer
    from nbformat.v4 import new_notebook, new_code_cell, new_markdown_cell
    from nbconvert import export, get_exporter
    from nbconvert.exporters.python import PythonExporter    
    from toolz.curried import compose, do, identity
    from types import MethodType

In [13]:
    def ipython2python(code):
        from nbconvert.filters.strings import ipython2python
        if any(map(code.startswith, '!%')):
            code = ipython2python(code)
        return code

    def ipython2python(code):
        from nbconvert.filters.strings import ipython2python
        if any(map(code.startswith, '!%')):
            code = ipython2python(code)
        return code

In [14]:
    def run_cell(ip, enter=[], exit=[]):
        """"""
        runner = process(enter, exit)
        def _run_cell(self, text, store_history=False, silent=False, shell_features=True):
            with runner(text) as code:
                if any(map(code.startswith, '!%')):
                    code = ipython2python(code)
                return InteractiveShell.run_cell(self, code, store_history, silent, shell_features)
                        
        ip.run_cell = MethodType(_run_cell, ip)

    def process(enter=[], exit=[]):
        @contextmanager
        def _run(code):
            for fn in enter:
                _, code = code, fn(code)
            yield code
            for fn in exit:
                fn()
        return _run

    def run_cell(ip, enter=[], exit=[]):
        """"""
        runner = process(enter, exit)
        def _run_cell(self, text, store_history=False, silent=False, shell_features=True):
            with runner(text) as code:
                return InteractiveShell.run_cell(self, ipython2python(code), store_history, silent, shell_features)
        ip.run_cell = MethodType(_run_cell, ip)

    def process(enter=[], exit=[]):
        @contextmanager
        def _run(code):
            for fn in enter:
                _, code = code, fn(code)
            yield code
            for fn in exit:
                fn()
        return _run

In [4]:
    class Weave(Renderer):
        """A mistune.Renderer to accumulate lines of code in a Markdown document."""
        code = """"""
        inline_code = True
        inline_indent = True
        
        def block_code(self, code, lang=None):
            self.code += code + '\n'
            return super(Weave, self).block_code(code, lang)

        def codespan(self, code):
            """Weave inline code references
            """
            if self.inline_code:
                if self.inline_indent:
                    self.code += ' '*self.indents(self.code) + code + '\n'
                else:
                    self.code += code + '\n'
            return super(Weave, self).codespan(code)
        
        @staticmethod
        def indents(code):
            """Determine the indent length of the last line."""
            if code:
                str = list(filter(lambda s: s.strip(), code.splitlines()))
                if str:  
                    return len(str[-1])-len(str[-1].lstrip())
            return 0


    class Tangle(Markdown):
        """A mistune.Markdown processor for literate programming."""
        def render(self, text, **kwargs):
            """Render ingests Markdown tests and returns executable portions"""
            text=self.rstrip(text)
            self.renderer.code = """"""
            super(Tangle, self).render(text, **kwargs)
            data =  '\n'.join(str.rstrip('    '+s) for s in self.renderer.code.splitlines())
            if not data.strip(): return """"""
            code = PythonExporter().from_notebook_node(
                new_notebook(cells=[new_code_cell(data)]))[0]
            return code

        @staticmethod
        def rstrip(text):
            """Remove whitespace for diffing later."""
            return '\n'.join(map(str.rstrip, map(str, text.splitlines())))

In [5]:
    literate = Tangle(renderer=Weave(), escape=False)

In [6]:
    def file_url_rules(code):
        from IPython import display
        if len(code.splitlines()) is 1 and fnmatch(code, 'http*://*'):
            return display.IFrame(code, width=600, height=400)
        try:
            path = __import__('pathlib').Path(code)
            if path.is_file(): 
                return display.Markdown(filename=str(path))
        except OSError:
            pass

In [7]:
    @do
    def weave(code):        
        if code.splitlines() and code.splitlines()[0].strip():
            output = file_url_rules(code) 
            display.display(
                output or __import__('IPython').display.Markdown(code))

In [8]:
    class Execute(__import__('nbconvert').preprocessors.execute.ExecutePreprocessor):
        tangle = staticmethod(literate.render)
        def run_cell(self, code, index=0):
            code.source = ipython2python(self.tangle(code.source))
            return super().run_cell(code, index)

In [9]:
    def load_ipython_extension(ip=get_ipython()):
        run_cell(ip, [weave, literate.render])
        loader(literate.render)
        
    def unload_ipython_extension(ip=get_ipython()):
        unload(['ipynb'])
        ip.run_cell = InteractiveShell.run_cell

In [10]:
    exporter = get_exporter('script')

    def nb_to_python(callable, nb):
        new = []
        for cell in nb.cells:
            if cell['cell_type'] == 'code':
                cell.source = callable(
                    '\n'.join(cell.source) if isinstance(cell.source, list) else cell.source)
                new.append(cell)
            nb.cells = new
        return export(exporter, nb)[0].encode('utf-8')

    def loader(render):
        def wrapped(self, path):
            nonlocal render
            with open(self.path) as f:
                out = nb_to_python(render, __import__('nbformat').read(f, 4))
            return out
        return unload(['ipynb']) or finder('ipynb')(wrapped)